# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences""" #permet de tokenizer des "doubles" mots, par ex "premier" + "ministre" va être considéré comme un token "premier ministre"
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

15850647

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

488

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

#imprime une liste de n-grammes, qu'on répère car ils sont séparés par des _

In [13]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads -> si la machine est sur le point d'exploser, diminuer
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.

# pour tester: petite fenêtre de mots, peu de workers, haut min_count, peu d'epoch -> puis améliorer itérativement le modèle

)

CPU times: total: 9min 58s
Wall time: 1h 1min 47s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [15]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [16]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [17]:
model.wv["ministre"] #si erreur, vérifier si le mot apparaît bien dans la sélection (surtout si on garde juste des mots très fréquents)

array([ 5.4153171e+00, -3.0359111e+00,  6.7417746e+00,  8.3304052e+00,
       -4.0960083e+00, -4.8396215e+00,  2.2318828e+00, -3.6771955e+00,
       -2.5350341e-01, -4.2664180e+00,  4.9434962e+00,  8.1468057e-03,
        4.8795958e+00, -8.5879236e-01,  7.7189426e+00,  2.6066208e+00,
       -3.3467281e+00, -9.8050863e-01,  5.2193060e+00, -5.3982735e+00,
       -3.9901528e+00, -5.6427498e+00,  2.6319098e+00, -6.0834961e+00,
        3.3287213e+00, -6.1779938e+00,  7.1288228e-01, -5.2359276e+00,
       -4.7287685e-01, -6.5384541e+00, -4.8574948e+00, -5.8479595e-01],
      dtype=float32)

### Calculer la similarité entre deux termes

In [18]:
model.wv.similarity("ministre", "roi")

0.68329084

### Chercher les mots les plus proches d'un terme donné

In [19]:
model.wv.most_similar("ministre", topn=10)

[('premier_ministre', 0.9209662675857544),
 ('ministre_des_finances', 0.8947284817695618),
 ('chancelier', 0.8863394260406494),
 ('ministere', 0.8832071423530579),
 ('rapporteur', 0.8725969195365906),
 ('gouverneur_general', 0.870328962802887),
 ('president', 0.8666937351226807),
 ('ministro', 0.8513811230659485),
 ('ancien_ministre', 0.8500110507011414),
 ('gouverneur', 0.8441176414489746)]

### Faire des recherches complexes à travers l'espace vectoriel

In [20]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))
    #positive = proche, negative=éloigné

[('berlin', 0.8118323087692261), ('new_york', 0.7778382897377014), ('rome', 0.7551426291465759), ('tokio', 0.7545449733734131), ('strasbourg', 0.7538933157920837), ('alger', 0.7509472966194153), ('vienne', 0.750045657157898), ('shanghai', 0.7497210502624512), ('casablanca', 0.7487617135047913), ('teheran', 0.7463179230690002)]
